In [1]:
! pip install torch scikit-learn imblearn matplotlib seaborn numpy pennylane umap-learn joblib

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import Counter
from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import label_binarize
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pennylane as qml
import umap
import joblib
import pandas as pd
import os

Could not save font_manager cache [Errno 13] Permission denied: 'C:\\Users\\91720\\.matplotlib\\fontlist-v390.json.matplotlib-lock'
d:\QCNN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# ---------------------- LOAD LABELS ----------------------
labels_df = pd.read_csv("experiment-ii_labels.csv")
base_dir = "experiment-ii"
file_paths, unique_labels = [], []
for _, row in labels_df.iterrows():
    full_path = os.path.join(base_dir, row["Subject"], row["Mat_Type"], row["File_Name"])
    if os.path.exists(full_path):
        file_paths.append(full_path)
        unique_labels.append(row["Label"])
label_mapping = {label: idx for idx, label in enumerate(sorted(set(unique_labels)))}
labels = [label_mapping[lbl] for lbl in unique_labels]
# ---------------------- LOAD MATRIX FILES FROM .TXT ----------------------
X = []
for path in file_paths:
    try:
        matrix = np.loadtxt(path)  # Assumes whitespace- or tab-delimited .txt files
        X.append(matrix.flatten())  # Flatten 2D matrix to 1D
    except Exception as e:
        print(f"Failed to load {path}: {e}")
X = np.array(X)
Y = np.array(labels)

In [9]:
# 1. Preprocessing
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
# Autoencoder
class AutoEncoder(nn.Module):
    def __init__(self, input_size, encoded_size):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128), nn.ReLU(),
            nn.Linear(128, encoded_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_size, 128), nn.ReLU(),
            nn.Linear(128, input_size), nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, x):
        return self.encoder(x)

input_size = X_scaled.shape[1]
encoded_size = 50

autoencoder = AutoEncoder(input_size, encoded_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [13]:
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
for epoch in range(150):
    optimizer.zero_grad()
    reconstructed = autoencoder(X_tensor)
    loss = criterion(reconstructed, X_tensor)
    loss.backward()
    optimizer.step()

X_encoded = autoencoder.encode(X_tensor).detach().numpy()

In [14]:
# 3. UMAP for further dimensionality reduction (to match qubit count)
n_qubits = 11
umap_model = umap.UMAP(n_components=n_qubits, random_state=42)
X_umap = umap_model.fit_transform(X_encoded)

joblib.dump(umap_model, "umap5_model.pkl")

d:\QCNN\.venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\QCNN\.venv\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


['umap5_model.pkl']

In [15]:
# 4. Handle class imbalance using SMOTE
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_umap, Y)
# Print class distribution after SMOTE
label_counts = Counter(y_resampled)
print("Class distribution after SMOTE:")
for label, count in label_counts.items():
    print(f"Label {label}: {count} samples")

Class distribution after SMOTE:
Label 2: 271 samples
Label 1: 271 samples
Label 0: 271 samples


In [16]:
class PostureDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [17]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
train_dataset = PostureDataset(X_train, y_train)
test_dataset = PostureDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [18]:
# 6. PennyLane Quantum Circuit Definition
dev = qml.device("default.qubit", wires=n_qubits)

In [40]:
def QuantumConvLayer(inputs, weights):
    if not isinstance(inputs, torch.Tensor):
        inputs = torch.tensor(inputs, dtype=torch.float32)

    num_qubits = inputs.shape[0]
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev, interface="torch")
    def circuit(x, w):
        qml.AngleEmbedding(x, wires=range(num_qubits), rotation='Y')
        for l in range(w.shape[0]):
            for i in range(num_qubits):
                qml.RY(w[l, i, 0], wires=i)
                qml.RZ(w[l, i, 1], wires=i)
            # Entanglement ring
            for i in range(num_qubits - 1):
                qml.CNOT(wires=[i, i + 1])
            qml.CNOT(wires=[num_qubits - 1, 0])
            # Optional deeper entanglement
            for i in range(num_qubits):
                qml.CRX(w[l, i, 0], wires=[i, (i + 2) % num_qubits])
        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    return circuit(inputs, weights)


In [41]:
def QuantumPoolingLayer(inputs):
    if not isinstance(inputs, torch.Tensor):
        inputs = torch.tensor(inputs, dtype=torch.float32)

    num_qubits = inputs.shape[0]
    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev, interface="torch")
    def circuit(x):
        qml.AngleEmbedding(x, wires=range(num_qubits), rotation='Y')
        for i in range(0, num_qubits - 1, 2):
            qml.CRY(np.pi / 2, wires=[i, i + 1])
            qml.CZ(wires=[i, i + 1])
        for i in range(num_qubits):
            qml.Hadamard(wires=i)
            qml.S(wires=i)
        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    return circuit(inputs)


In [42]:
class QCNN(nn.Module):
    def __init__(self, num_qubits=11, num_layers=3, num_classes=3):
        super(QCNN, self).__init__()
        self.num_qubits = num_qubits
        self.num_layers = num_layers

        # 3 sets of weights for 3 conv layers
        self.q_params1 = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits, 2))
        self.q_params2 = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits, 2))
        self.q_params3 = nn.Parameter(0.01 * torch.randn(num_layers, num_qubits, 2))

        # Classical Dense Layers (3 stages)
        self.fc1 = nn.Linear(num_qubits, 128)
        self.norm1 = nn.LayerNorm(128)

        self.fc2 = nn.Linear(128, 64)
        self.norm2 = nn.LayerNorm(64)

        self.fc3 = nn.Linear(64, 32)
        self.norm3 = nn.LayerNorm(32)

        self.out = nn.Linear(32, num_classes)

    def forward(self, x, return_quantum_features=False):
        x = x.view(x.shape[0], -1)
        quantum_outputs = []

        for sample in x:
            # Conv + Pool 1
            conv1 = QuantumConvLayer(sample, self.q_params1)
            pool1 = QuantumPoolingLayer(torch.tensor(conv1, dtype=torch.float32))

            # Conv + Pool 2
            conv2 = QuantumConvLayer(pool1, self.q_params2)
            pool2 = QuantumPoolingLayer(torch.tensor(conv2, dtype=torch.float32))

            # Conv + Pool 3
            conv3 = QuantumConvLayer(pool2, self.q_params3)
            pool3 = QuantumPoolingLayer(torch.tensor(conv3, dtype=torch.float32))

            quantum_outputs.append(torch.tensor(pool3, dtype=torch.float32))

        x = torch.stack(quantum_outputs).to(x.device)

        if return_quantum_features:
            return self.out(F.relu(self.fc3(F.relu(self.fc2(F.relu(self.fc1(x)))))), x)

        x = F.relu(self.norm1(self.fc1(x)))
        x = F.dropout(x, p=0.3)

        x = F.relu(self.norm2(self.fc2(x)))
        x = F.dropout(x, p=0.2)

        x = F.relu(self.norm3(self.fc3(x)))
        x = F.dropout(x, p=0.1)

        return self.out(x)


In [43]:
num_classes = len(np.unique(y_resampled))
num_qubits=11
model = QCNN(num_classes=num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [44]:
# Trackers
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

n_epochs = 50

for epoch in range(n_epochs):
    # ---------- TRAINING ----------
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    avg_train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train
    train_losses.append(avg_train_loss)
    train_accuracies.append(train_accuracy)

    # ---------- VALIDATION ----------
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_val += (predicted == labels).sum().item()
            total_val += labels.size(0)

    avg_val_loss = val_loss / total_val
    val_accuracy = correct_val / total_val
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)

    # ---------- LOG ----------
    print(f"Epoch [{epoch+1}/{n_epochs}]")
    print(f"  Train    -> Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy*100:.2f}%")
    print(f"  Validate -> Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy*100:.2f}%\n")


Epoch [1/50]
  Train    -> Loss: 1.1192, Accuracy: 31.69%
  Validate -> Loss: 1.1041, Accuracy: 34.36%

Epoch [2/50]
  Train    -> Loss: 1.1042, Accuracy: 34.46%
  Validate -> Loss: 1.1314, Accuracy: 27.61%

Epoch [3/50]
  Train    -> Loss: 1.1081, Accuracy: 35.08%
  Validate -> Loss: 1.1148, Accuracy: 30.67%

Epoch [4/50]
  Train    -> Loss: 1.0909, Accuracy: 38.15%
  Validate -> Loss: 1.0943, Accuracy: 38.04%

Epoch [5/50]
  Train    -> Loss: 1.1035, Accuracy: 36.00%
  Validate -> Loss: 1.1009, Accuracy: 36.81%

Epoch [6/50]
  Train    -> Loss: 1.0884, Accuracy: 37.08%
  Validate -> Loss: 1.0990, Accuracy: 33.74%

Epoch [7/50]
  Train    -> Loss: 1.0990, Accuracy: 35.69%
  Validate -> Loss: 1.0725, Accuracy: 44.17%

Epoch [8/50]
  Train    -> Loss: 1.0882, Accuracy: 39.23%
  Validate -> Loss: 1.0800, Accuracy: 38.65%

Epoch [9/50]
  Train    -> Loss: 1.0682, Accuracy: 40.62%
  Validate -> Loss: 1.0493, Accuracy: 42.94%

Epoch [10/50]
  Train    -> Loss: 1.0499, Accuracy: 42.31%
  Val

In [45]:
# ---------- TESTING ----------
model.eval()
test_loss = 0.0
correct_test = 0
total_test = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_test += (predicted == labels).sum().item()
        total_test += labels.size(0)

avg_test_loss = test_loss / total_test
test_accuracy = correct_test / total_test

print("---------- FINAL TEST RESULTS ----------")
print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


---------- FINAL TEST RESULTS ----------
Test Loss: 0.9621
Test Accuracy: 48.47%


In [46]:
# Assume `model` is your trained QCNN instance
torch.save(model.state_dict(), "qcnn5_model.pth")
print("✅ Trained QCNN model  saved as 'qcnn5_model.pth'")

✅ Trained QCNN model  saved as 'qcnn5_model.pth'


In [ ]:
# -------- Loss & Accuracy Curves --------
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss Curve")
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy Curve")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Collect all predictions and true labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Classification report
print("---------- CLASSIFICATION REPORT ----------")
print(classification_report(all_labels, all_preds))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.tight_layout()
plt.show()


In [2]:
# Prediction function
def predict_from_file(file_path):
    if not os.path.exists(file_path):
        print("File does not exist.")
        return

    # Load and visualize pressure matrix
    matrix = np.loadtxt(file_path)
    plt.imshow(matrix, cmap='hot', interpolation='nearest')
    plt.title("Pressure Matrix Heatmap")
    plt.colorbar()
    plt.show()

    # Preprocess: scale, flatten, reduce
    matrix_flat = matrix.flatten().reshape(1, -1)

    # Now check shape match
    if matrix_flat.shape[1] != scaler.n_features_in_:
        raise ValueError(f"Expected {scaler.n_features_in_} features, but got {matrix_flat.shape[1]}")

    matrix_scaled = scaler.transform(matrix_flat)

    # Load trained UMAP model
    umap_model = joblib.load("umap5_model.pkl")
    matrix_umap = umap_model.transform(matrix_scaled)
    input_tensor = torch.tensor(matrix_umap, dtype=torch.float32)

    # Load model
    model = QCNN(11,6, num_classes=3)  # Update num_classes if needed
    model.load_state_dict(torch.load("qcnn3_model.pth", map_location=torch.device('cpu')))
    model.eval()

    # Predict
    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).item()

    # Define label mapping manually or load from file if saved
    label_mapping = {label: idx for idx, label in enumerate(sorted(labels_df['Label'].unique()))}
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}

    print(f"Predicted Posture: {reverse_label_mapping.get(pred, 'Unknown')}")

predict_from_file("D:/QCNN/experiment-ii/S1/Sponge_Mat/Matrix_Sponge_C1.txt")

NameError: name 'os' is not defined